In [ ]:
import csv
import pandas as pd

df = pd.DataFrame()
# Load stops.txt, routes.txt, and stop_times.txt
with open('gtfs/stops.txt', 'r', encoding='utf-8') as stops_file, \
     open('gtfs/routes.txt', 'r', encoding='utf-8') as routes_file, \
     open('gtfs/stop_times.txt', 'r', encoding='utf-8') as stop_times_file:
    stops_reader = csv.DictReader(stops_file)
    routes_reader = csv.DictReader(routes_file)
    stop_times_reader = csv.DictReader(stop_times_file)

    # Create a mapping of stops to routes
    stop_routes = {}

    # Parse routes.txt to get route types
    route_types = {}
    for row in routes_reader:
        route_id = row['\ufeffroute_id']
        route_type = row['route_type']
        route_types[route_id] = route_type

    # Parse stop_times.txt to record routes that stop at each stop
    for row in stop_times_reader:
        trip_id = row['\ufefftrip_id']
        stop_id = row['stop_id']

        if stop_id not in stop_routes:
            stop_routes[stop_id] = []

        for route_id in route_types:
            if route_id in trip_id:
                if route_id not in stop_routes[stop_id]:
                    stop_routes[stop_id].append(route_id)
                break

    # Associate route types with stops
    stop_types = {}
    for stop_id, route_ids in stop_routes.items():
        types = []
        for route_id in route_ids:
            route_type = route_types.get(route_id)
            if "N" in route_id: # Night bus
                route_type = '2'
            if route_type and route_type not in types:
                types.append(route_type)

        stop_types[stop_id] = types

    # Print the types of transport served at each stop
    for row in stops_reader:
        stop_id = row['stop_id']
        stop_name = row['stop_name']
        types = stop_types.get(stop_id)
        if types:
            print(f"{stop_name}: {', '.join(types)}")
        else:
            print(f"{stop_name}: No transport types found.")
        df = df.append({'stop_id' : stop_id, 'stop_type' : ', '.join(types)}, ignore_index = True)


In [54]:
df.to_csv("stop_types.txt", index=False)